In [1]:
from langchain.chains import GraphCypherQAChain
from langchain_community.graphs import Neo4jGraph
from langchain_openai import ChatOpenAI

import os
from dotenv import load_dotenv
load_dotenv()

True

In [ ]:
OPENAI_API_KEY = os.environ.get("OPENAI_API_KEY")
USERNAME = os.environ.get("USERNAME")
PASSWORD = os.environ.get("PASSWORD")
URL = os.environ.get("URL")
llm = ChatOpenAI(model_name="gpt-5-mini", temperature=0)

In [3]:
graph = Neo4jGraph(url=URL, username=USERNAME, password=PASSWORD)

C:\Users\faeqs\AppData\Local\Temp\ipykernel_23688\3113965413.py:1: LangChainDeprecationWarning: The class `Neo4jGraph` was deprecated in LangChain 0.3.8 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-neo4j package and should be used instead. To use it run `pip install -U :class:`~langchain-neo4j` and import as `from :class:`~langchain_neo4j import Neo4jGraph``.
  graph = Neo4jGraph(url=URL, username=USERNAME, password=PASSWORD)


In [9]:
chain = GraphCypherQAChain.from_llm(
    llm=llm,
    graph=graph, 
    verbose=True, 
    allow_dangerous_requests=True,
    return_intermediate_steps=True  # 생성된 Cypher 쿼리와 중간 결과 확인
)

In [ ]:
# from langchain_google_genai import ChatGoogleGenerativeAI
# Cypher 쿼리 생성은 OpenAI 모델, 최종 답변은 Gemini 모델 사용
# cypher_llm = ChatOpenAI(model="gpt-4.1-mini", temperature=0.0)
# qa_llm = ChatGoogleGenerativeAI(model="gemini-2.0-flash", temperature=0.0)

cypher_chain = GraphCypherQAChain.from_llm(
    # cypher_llm=cypher_llm,
    # qa_llm=qa_llm,
    llm=llm,
    graph=graph, 
    allow_dangerous_requests=True,
    verbose=True,
    return_intermediate_steps=True,  # 생성된 Cypher 쿼리와 중간 결과 확인
    # return_direct=True
)

answer = cypher_chain.invoke({"query": "Bella Roma 레스토랑의 주중 영업시간을 알려줘."})
answer




> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (r:Restaurant {name: "Bella Roma"}) RETURN r.hoursWeekday AS hoursWeekday;
Full Context:
[{'hoursWeekday': '11:00-22:00'}]

> Finished chain.


{'query': 'Bella Roma 레스토랑의 주중 영업시간을 알려줘.',
 'result': 'Bella Roma 레스토랑의 주중 영업시간은 11:00-22:00입니다.',
 'intermediate_steps': [{'query': 'MATCH (r:Restaurant {name: "Bella Roma"}) RETURN r.hoursWeekday AS hoursWeekday;'},
  {'context': [{'hoursWeekday': '11:00-22:00'}]}]}

In [ ]:
# 테스트용 질문 목록
questions = [
    "Bella Roma 레스토랑의 영업시간을 알려주세요.",
    "까르보나라 파스타의 가격은 얼마인가요?",
    "주방장 이름이 뭐예요?",
    "티라미수를 구매한 이력이 있는 사용자들이 남긴 리뷰를 모두 알려줘",
    "2024년 8월에 가장 많이 팔린 메뉴는 무엇인가요?",
    "글루텐 알레르기가 있는 고객이 먹을 수 없는 메뉴는 무엇인가요?",
    "2024년 상반기(1~6월)에 긍정적인 리뷰(4점 이상)를 남긴 고객들이 가장 많이 주문한 메뉴 카테고리는 무엇인가요?",
    "‘가격’이라는 키워드가 포함된 리뷰들의 평균 평점은 얼마이며, 이 리뷰를 남긴 사용자들은 주로 어떤 메뉴를 주문했나요?"
]

# 질문들을 chain에 넣고 결과 출력
for idx, q in enumerate(questions, 1):
    print(f"\n{'='*30}\nQ{idx}. {q}\n{'-'*30}")
    answer = chain.run(q)
    print(f"A{idx}. {answer}\n")


Q1. Bella Roma 레스토랑의 영업시간을 알려주세요.
------------------------------


> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (r:Restaurant {name: "Bella Roma"}) RETURN r.hoursWeekday AS hoursWeekday, r.hoursWeekend AS hoursWeekend;
Full Context:
[{'hoursWeekday': '11:00-22:00', 'hoursWeekend': '11:00-23:00'}]

> Finished chain.
A1. Bella Roma 레스토랑의 영업시간은 평일 11:00-22:00, 주말 11:00-23:00입니다.


Q2. 까르보나라 파스타의 가격은 얼마인가요?
------------------------------


> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (m:MenuItem {name: "까르보나라 파스타"}) RETURN m.name AS name, m.price AS price, m.currency AS currency;
Full Context:
[{'name': '까르보나라 파스타', 'price': 18000, 'currency': 'KRW'}]

> Finished chain.
A2. 까르보나라 파스타의 가격은 18,000원입니다.


Q3. 주방장 이름이 뭐예요?
------------------------------


> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (e:Employee)
WHERE toLower(e.role) CONTAINS '주방' OR toLower(e.role) CONTAINS 'chef' OR toLower(e.role) CONTAINS 'head'
RETURN DIST